In [1]:
#GPU information
!nvidia-smi

Mon May 31 12:17:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#Mounting the Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#Changing working directory
%cd /content/gdrive/My Drive/ML-in-colab/Sentiment_analysis_amazon_fine_food_review

/content/gdrive/My Drive/ML-in-colab/Sentiment_analysis_amazon_fine_food_review


In [4]:
#Installing Dependencies
!pip install tensorflow_text

     |████████████████████████████████| 4.3MB 8.3MB/s 


In [5]:
#Importing Liabries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import time

In [6]:
#Loading Data into DataFrame
data = pd.read_csv('data/preprocessed_reviews.csv')
data = data[:15000] # we are limiting ourself to 15k Reviews because of Computational Constraint
data.head()

,Text,Score
0,i recently purchased a bottle each of the suga...,0
1,this is an excellent gluten free alternative t...,1
2,my dog and i both love this product my vet eve...,1
3,your friends and family may envy you when you ...,1
4,these should have an age limit i do not think ...,0


In [7]:
def removeDup(df):
  if df.duplicated().any() == True:
    print("Found {} duplicated".format(df.duplicated().sum()))
    df.drop_duplicates(inplace=True, ignore_index=True)
    print("Duplicated removed")
    return df
  else:
    print("Duplicates not found")
    return df

data = removeDup(data)

Found 34 duplicated
Duplicated removed


In [8]:
#Checking and removing Null values if any

def removeNull(df):
  "This function will check Duplicated and remove if any "
  if df.isnull().any().any() == True:
    print("Found {} Null values".format(df.isnull().values.sum()))
    df.dropna(inplace=True)
    print("Null Values Dropped")
    return df
  else:
    print("No Null value found")
    return df

data = removeNull(data)

Found 2 Null values
Null Values Dropped


In [9]:
#Spliting tha data into train test

train_size = 0.668  #Spliting to get nearly 10k in training and 5k in testing
train_data, test_data = train_test_split(data, train_size= train_size, shuffle=True)
print("Length of Train Data:",len(train_data))
print("Length of Test Data:",len(test_data))

Length of Train Data: 9995
Length of Test Data: 4969


**MLP Model - bert Large as embedding layer**

In [10]:
#Importing Bert pretrained model from Tensorflow Hub
preprocess_model_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_encoder_model_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4'

precessing_layer = hub.KerasLayer( preprocess_model_url)
bert_layer = hub.KerasLayer( bert_encoder_model_url, trainable = False )

In [11]:
#Creating MLP Model

def normalModel(precessing_layer, bert_layer):
  input = tf.keras.layers.Input(shape=(), dtype=tf.string)
  encorder_input = precessing_layer(input)
  encorder_output = bert_layer(encorder_input)
  final_output = encorder_output['pooled_output']
  x = tf.keras.layers.Dense(1024, activation='relu')(final_output)
  x = tf.keras.layers.Dense(2048, activation='relu')(x)
  x = tf.keras.layers.Dense(2048, activation='relu')(x)
  x = tf.keras.layers.Dense(1024, activation='relu')(x)
  x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
  model = tf.keras.Model(input, x)

  model.compile(loss='BinaryCrossentropy' , optimizer = Adam(learning_rate=1e-5), metrics= 'BinaryAccuracy' )
  return model

**N.P - Bert large model is only used for converting our text to numerical form so that we can apply MLP here Which makes it a MLP model**

In [12]:
mlp_model = normalModel(precessing_layer, bert_layer)
mlp_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_word_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'encoder_outputs':  335141889   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [13]:
%%time

#Training our MLP Model
mlp_model_history = mlp_model.fit(x = train_data['Text'].to_numpy(), y = train_data['Score'].to_numpy(),batch_size = 32,
                    validation_data = (test_data['Text'].to_numpy(), test_data['Score'].to_numpy() ),
                               epochs=5)

Epoch 1/5
313/313 [==============================] - 267s 805ms/step - loss: 0.6787 - binary_accuracy: 0.5681 - val_loss: 0.6807 - val_binary_accuracy: 0.5506
Epoch 2/5
313/313 [==============================] - 251s 803ms/step - loss: 0.6375 - binary_accuracy: 0.6498 - val_loss: 0.6071 - val_binary_accuracy: 0.6981
Epoch 3/5
313/313 [==============================] - 251s 803ms/step - loss: 0.5897 - binary_accuracy: 0.6988 - val_loss: 0.5485 - val_binary_accuracy: 0.7519
Epoch 4/5
313/313 [==============================] - 251s 803ms/step - loss: 0.5392 - binary_accuracy: 0.7381 - val_loss: 0.5015 - val_binary_accuracy: 0.7744
Epoch 5/5
313/313 [==============================] - 251s 803ms/step - loss: 0.4962 - binary_accuracy: 0.7677 - val_loss: 0.4742 - val_binary_accuracy: 0.7802
CPU times: user 6min 22s, sys: 3min 3s, total: 9min 25s
Wall time: 21min 35s


**MLP model - Bert Small as Embedding layer**

In [14]:
preprocess_model_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_small_model_url = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'

precessing_layer = hub.KerasLayer( preprocess_model_url)
bert_small_layer = hub.KerasLayer( bert_small_model_url, trainable = False )

In [15]:
#Creating MLP model 

def small_model(precessing_layer, bert_layer):
  input = tf.keras.layers.Input(shape=(), dtype=tf.string)
  encorder_input = precessing_layer(input)
  encorder_output = bert_layer(encorder_input)
  final_output = encorder_output['pooled_output']
  x = tf.keras.layers.Dense(512, activation='relu')(final_output)
  x = tf.keras.layers.Dense(1024, activation='relu')(x)
  x = tf.keras.layers.Dense(1024, activation='relu')(x)
  x = tf.keras.layers.Dense(512, activation='relu')(x)
  x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
  model = tf.keras.Model(input, x)
  
  model.compile(loss='BinaryCrossentropy' , optimizer = Adam(learning_rate=1e-5), metrics= 'BinaryAccuracy' )
  return model

**N.P - Bert small model is only used for converting our text to numerical form so that we can apply MLP here Which makes it a MLP model**

In [16]:
bert_small_model = small_model(precessing_layer, bert_small_layer)
bert_small_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      {'input_word_ids': ( 0           input_2[0][0]                    
__________________________________________________________________________________________________
keras_layer_3 (KerasLayer)      {'sequence_output':  28763649    keras_layer_2[0][0]              
                                                                 keras_layer_2[0][1]              
                                                                 keras_layer_2[0][2]              
____________________________________________________________________________________________

In [17]:
%%time

#Training our MLP model
bert_small_history = bert_small_model.fit(x = train_data['Text'].to_numpy(), y = train_data['Score'].to_numpy(),batch_size = 32,
                    validation_data = (test_data['Text'].to_numpy(), test_data['Score'].to_numpy() ),
                               epochs=5)

Epoch 1/5
313/313 [==============================] - 40s 120ms/step - loss: 0.5747 - binary_accuracy: 0.7235 - val_loss: 0.4948 - val_binary_accuracy: 0.7627
Epoch 2/5
313/313 [==============================] - 37s 119ms/step - loss: 0.4678 - binary_accuracy: 0.7774 - val_loss: 0.4548 - val_binary_accuracy: 0.7909
Epoch 3/5
313/313 [==============================] - 37s 119ms/step - loss: 0.4449 - binary_accuracy: 0.7919 - val_loss: 0.4477 - val_binary_accuracy: 0.7977
Epoch 4/5
313/313 [==============================] - 37s 119ms/step - loss: 0.4335 - binary_accuracy: 0.7946 - val_loss: 0.4470 - val_binary_accuracy: 0.7961
Epoch 5/5
313/313 [==============================] - 37s 118ms/step - loss: 0.4254 - binary_accuracy: 0.7983 - val_loss: 0.4346 - val_binary_accuracy: 0.8014
CPU times: user 2min 57s, sys: 36.4 s, total: 3min 33s
Wall time: 3min 8s


**Bert Large Trainable model**

In [18]:
#Importing Bert pretrained model from Tensorflow Hub
preprocess_model_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_encoder_model_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4'

precessing_layer = hub.KerasLayer( preprocess_model_url)
bert_layer = hub.KerasLayer( bert_encoder_model_url, trainable = True )

In [19]:
#Creating Bert model

def bert_Model(precessing_layer, bert_layer):
  input = tf.keras.layers.Input(shape=(), dtype=tf.string)
  encorder_input = precessing_layer(input)
  encorder_output = bert_layer(encorder_input)
  final_output = encorder_output['pooled_output']
  x = tf.keras.layers.Dense(1, activation='sigmoid')(final_output)
  model = tf.keras.Model(input, x)

  model.compile(loss='BinaryCrossentropy' , optimizer = Adam(learning_rate=1e-5), metrics= 'BinaryAccuracy' )
  return model

In [20]:
bert_model = bert_Model(precessing_layer, bert_layer)
bert_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_4 (KerasLayer)      {'input_word_ids': ( 0           input_3[0][0]                    
__________________________________________________________________________________________________
keras_layer_5 (KerasLayer)      {'encoder_outputs':  335141889   keras_layer_4[0][0]              
                                                                 keras_layer_4[0][1]              
                                                                 keras_layer_4[0][2]              
____________________________________________________________________________________________

In [21]:
%%time

#Training our Bert Large Model
bert_model_history = bert_model.fit(x = train_data['Text'].to_numpy(), y = train_data['Score'].to_numpy(),batch_size = 16,
                    validation_data = (test_data['Text'].to_numpy(), test_data['Score'].to_numpy() ),
                               epochs=3)


Epoch 1/3
625/625 [==============================] - 632s 977ms/step - loss: 0.2468 - binary_accuracy: 0.8961 - val_loss: 0.1819 - val_binary_accuracy: 0.9288
Epoch 2/3
625/625 [==============================] - 609s 975ms/step - loss: 0.1164 - binary_accuracy: 0.9587 - val_loss: 0.1819 - val_binary_accuracy: 0.9278
Epoch 3/3
625/625 [==============================] - 609s 975ms/step - loss: 0.0639 - binary_accuracy: 0.9784 - val_loss: 0.2033 - val_binary_accuracy: 0.9362
CPU times: user 18min 14s, sys: 8min 1s, total: 26min 15s
Wall time: 30min 51s
